# Team Project 3 - Solving a System of Linear Equations

In this project, we compare the performance of three different methods of solving a system of linear equations. The lesson that I hope you may get is that finding an efficient method is not an unneccessary complication.

#### 1. (10 pts) Create a function randmat(n) which returns a random square matrix constructed as the following recipe. 
<ul>
    <li>The size of the matrix is $n \times n$.</li>
    <li>Each off-diagonal entry ($a_{ij}$ where $i \ne j$) is a random number in $[0, 1)$. A random number can be constructed by the random method (see <a href="https://docs.scipy.org/doc/numpy/reference/routines.random.html">here</a>).</li>
    <li>A diagonal entry $a_{ii}$ is a random number in $[n, n+1)$. (This condition guarantees that the matrix $(a_{ij})$ is strictly diagonally dominant, hence invertible.)</li>
</ul>

#### And create a function randvec(n) which returns an $n$-dimensional random vector whose entries are random numbers in $[0, 100)$. 

In [1]:
import numpy as np

def randmat(n):
    arr = [[0 for i in range(n)] for j in range(n)]
    for i in range(n):
        for j in range(n):
            if i == j:
                arr[i][i] = np.random.random() + n
            else:
                arr[i][j] = np.random.random()
    return arr

def randvec(n):
    vec = []
    for i in range(n):
        x = np.random.uniform(0,100)
        vec.append(x)
    return vec

n = np.random.randint(1, 5)

#### 2. (10 pts) Create a function GaussElim(A, b) which solves a system of linear equations $Ax = b$ by using Gaussian Elimination with the partial pivoting.

In [2]:
import numpy as np

def GaussElim(A,b):
    n = len(A)
    for i in range(0,n):
        A[i].append(b[i])
    for i in range(0,n):
        maxEl = np.abs(A[i][i])
        index = i
    #find max below the diagonal
        for k in range(i+1, n):
            if (np.abs(A[k][i]) > maxEl):
                maxEl = np.abs(A[k][i])
                index = k
        for k in range(i, n+1):
            temp = A[index][k]
            A[index][k] = A[i][k]
            A[i][k] = temp
        for k in range(i+1, n):
            num = -(A[k][i]/A[i][i])
            for j in range(i, n+1):
                if i == j:
                    A[k][j] = 0
                else:
                    A[k][j] += num * A[i][j]

    #solve upper-triangular matrix equation
    x = [0 for i in range(n)]
    for i in range(n-1,-1,-1):
        x[i] = (A[i][n])/(A[i][i])
        for k in range(i-1, -1,-1):
            A[k][n] -= (A[k][i])*(x[i])
    return x


A = [[2,-1,0], [-1,3,-1], [0,-1,2]]
b = [1,8,-5]


print(GaussElim(A,b))


[2.0, 3.0, -0.9999999999999998]


#### 3. (10 pts) Create a function Jacobi(A, b, err) which solves a system of linear equations $Ax = b$ by using Jacobi interation method. Set $x^{(0)} = \vec{0}$. We stop the iteration when the estimation of the error $||x^{(k)} - x^{(k-1)}||_\infty$ is less than err or $k = 1000$. (Here $x^{(k)}$ is the $k$-th output of the iteration).

In [3]:
def Jacobi(A, b, err):
    # Find D
    D = [[0 for i in range(len(A))] for j in range(len(A))]
    for i in range(len(A)):
        D[i][i] = A[i][i]
    
    # Find R = L + U
    R = [[0 for i in range(len(A))] for j in range(len(A))]
    for i in range(len(A)):
        for j in range(len(A)):
            R[i][j] = D[i][j] - A[i][j]
   
    # Create initial guess x-vector        
    x_0 = [0 for i in range(len(A))]   
    
    #Find D^-1
    D_inv = [[0 for i in range(len(A))] for j in range(len(A))]
    for i in range(len(A)):
        D_inv[i][i] = 1/(A[i][i])

    # Do the iterations
 
    x_1 = [0 for i in range (len(A))]
    
    k = 1
    while True:
        y = np.matmul(R,x_0)
        n = np.matmul(D_inv,y)
        q = np.matmul(D_inv,b)
        for i in range(len(A)):
            x_1[i] = n[i] + q[i]
        k += 1
        if (findnorm(x_1, x_0) < err or k == 1000):
            break
        else:
            for i in range (len(A)):
                x_0[i] = x_1[i]
    return x_1
        
                 
def findnorm(x_1, x_0):
    max = np.abs(x_1[0]-x_0[0])
    for i in range(len(x_0)):
        diff = np.abs(x_1[i] - x_0[i])
        if diff > max:
            max = diff  
    return max

A = [[2,-1,0], [-1,3,-1], [0,-1,2]]
b = [1,8,-5]


print(Jacobi(A,b,0.000001))

[1.9999998954624207, 2.9999993727745253, -1.0000001045375793]


#### 4. (10 pts) Create a function GaussSeidel(A, b, err) which solves a system of linear equations $Ax = b$ by using Gauss-Seidel interation method. Set $x^{(0)} = \vec{0}$. We stop the iteration when the estimation of the error $||x^{(k)} - x^{(k-1)}||_\infty$ is less than err or $k = 1000$. (Here $x^{(k)}$ is the $k$-th output of the iteration).

In [4]:
import numpy as np

def GaussSeidel (A,b, err):
    # Find D
    D = [[0 for i in range(len(A))] for j in range(len(A))]
    for i in range(len(A)):       
         D[i][i] = A[i][i]
    
    # Find R = L + U
    R = [[0 for i in range(len(A))] for j in range(len(A))]
    for i in range(len(A)):
        for j in range(len(A)):
            R[i][j] = A[i][j] - D[i][j]
            
    # Find D^-1
    D_inv = [[0 for i in range(len(A))] for j in range(len(A))]
    for i in range(len(A)):
            D_inv[i][i] = 1/(A[i][i])
                
    # Create initial guess x-vector        
    x_0 = [0.0 for i in range(len(A))] 
    
    x = [0.0 for i in range(len(A))]
    
    #Do iterations
    
    k = 1
    z = [0.0 for i in range(len(A))]
    while True:
        for i in range(len(A)):
            z[i] = x_0[i]
            x[i] = D_inv[i][i] * (b[i]-np.matmul(R[i],x_0))
            x_0[i] = x[i] 
        if (findnorm(z, x) < err or k == 1000):
            break
        else:
            k += 1
    return x
                

def findnorm(x_0, x):
    max = np.abs(x[0]-x_0[0])
    for i in range(len(x_0)):
        diff = np.abs(x[i] - x_0[i])
        if diff > max:
            max = diff  
    return max

A = [[2,-1,0], [-1,3,-1], [0,-1,2]]
b = [1,8,-5]


print(GaussSeidel(A,b,0.00001))

[1.9999957662280476, 2.9999971774853647, -1.0000014112573177]


#### 5. (10 pts) For $n = 100, 200, 300, \cdots , 1000$, create a random $n \times n$ matrix $A$ and a random $n$-dimensional vector $b$. Solve the system of linear equations $Ax = b$ by using GaussElim(A, b), Jacobi(A, b, err), and GaussSeidel(A, b, err). Use $10^{-6}$ for the error tolerance. Record the excution time for each method. Plot the graph of the excution time for those three methods on the same plane.

For the computation of the excution time, you may use the following method:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time

list1 = []
list2 = []
list3 = []

#This code takes like a minute or two, please be patient :)

for n in range(100,1000,100):
 
    #Create random vector(n x 1) and matrix (n x n)
    A = randmat(n)
    b = randvec(n)
    err = 10 ** (-6)
    
    start = time.time()
    GaussElim(A,b)
    end = time.time()
    
    list1.append(end - start)
    
    start = time.time()
    Jacobi(A,b,err)
    end = time.time()
    
    list2.append(end - start)
    
    start = time.time()
    GaussSeidel(A,b,err)
   
    end = time.time()

    list3.append(end - start)
    
x1 = np.linspace(100,1000,9)
y1 = list1

x2 = np.linspace(100,1000,9)
y2 = list2

x3 = np.linspace(100,1000,9)
y3 = list3


plt.plot(x1, y1, label = 'GaussElim')
plt.plot(x2, y2, label = 'Jacobi')
plt.plot(x3, y3, label = 'GaussSeidel')

plt.xlabel('Matrix Size (n x n)') 
plt.ylabel('Runtime') 

plt.title("Execution Times: 3 Methods")
plt.legend()
plt.show()

plt.plot(x2, y2, label = 'Jacobi')
plt.plot(x3, y3, label = 'GaussSeidel')

plt.xlabel('Matrix Size (n x n)') 
plt.ylabel('Runtime') 

plt.title("Execution Times: 2 Better Methods")
plt.legend()
plt.show()